In [1]:
import sys
sys.path.append("..")

import torch
from torch.optim import AdamW
from transformers import T5Tokenizer
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration

from datasets import load_dataset
from transformers import get_linear_schedule_with_warmup

from scripts.global_vars import (
    DEVICE, 
    BATCH_SIZE, 
    MAX_TURNS,
    MODEL_NAME,
    USE_TRAINED_MODEL,
    MAX_LENGTH_ENCODER_ACTION, 
    MAX_LENGTH_DECODER_ACTION
)

from scripts.utils import find_zero_percentage
from scripts.pytorch.training import train_model
from scripts.pytorch.inference import inference_model
from scripts.preprocessing.action import ActionDataset

In [2]:
dataset = load_dataset("multi_woz_v22", trust_remote_code=True)

train_data = dataset['train']
val_data = dataset['validation']

In [3]:
tokenizer = T5Tokenizer.from_pretrained(
    legacy=True,
    pretrained_model_name_or_path=MODEL_NAME
)

train_action_dataset = ActionDataset(
    data=dataset['train'],
    tokenizer=tokenizer,
    max_turns=MAX_TURNS,
    max_output_len=MAX_LENGTH_DECODER_ACTION,
    max_input_len=MAX_LENGTH_ENCODER_ACTION
)

valid_action_dataset = ActionDataset(
    data=dataset['validation'],
    tokenizer=tokenizer,
    max_turns=MAX_TURNS,
    max_output_len=MAX_LENGTH_DECODER_ACTION,
    max_input_len=MAX_LENGTH_ENCODER_ACTION
)

train_loader_action = DataLoader(train_action_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader_action = DataLoader(valid_action_dataset, batch_size=BATCH_SIZE)

batch = next(iter(train_loader_action))
print("Inputs IDs shape:", batch['encoder_input_ids'].shape)
print("Action IDs shape:", batch['decoder_input_ids'].shape)

Processing dialogues: 100%|██████████| 1000/1000 [00:00<00:00, 2022.11it/s]


Inputs IDs shape: torch.Size([256, 64])
Action IDs shape: torch.Size([256, 32])


In [4]:
train_encoder_zero = find_zero_percentage(train_loader_action, "encoder_input_ids", MAX_LENGTH_ENCODER_ACTION)
train_decoder_zero = find_zero_percentage(train_loader_action, "decoder_input_ids", MAX_LENGTH_DECODER_ACTION)
valid_encoder_zero = find_zero_percentage(valid_loader_action, "encoder_input_ids", MAX_LENGTH_ENCODER_ACTION)
valid_decoder_zero = find_zero_percentage(valid_loader_action, "decoder_input_ids", MAX_LENGTH_DECODER_ACTION)

In [5]:
print(
    "Train Encoder Zero Percentage:", train_encoder_zero * 100,
    "\nTrain Decoder Zero Percentage:", train_decoder_zero * 100,
    "\nValid Encoder Zero Percentage:", valid_encoder_zero * 100,
    "\nValid Decoder Zero Percentage:", valid_decoder_zero * 100
)

Train Encoder Zero Percentage: 33.90093445777893 
Train Decoder Zero Percentage: 27.37710475921631 
Valid Encoder Zero Percentage: 32.69616961479187 
Valid Decoder Zero Percentage: 26.663359999656677


In [6]:
num_epochs = 5
num_training_steps = len(train_loader_action) * num_epochs
num_warmup_steps = num_training_steps // 10

action_model = T5ForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path=MODEL_NAME
).to(DEVICE)

optimizer = AdamW(
    action_model.parameters(),
    lr=1e-3,
    eps=1e-8
)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)

In [7]:
if USE_TRAINED_MODEL:
    action_model = train_model(
        action_model,
        optimizer,
        scheduler,
        train_loader_action,
        valid_loader_action,
        num_epochs=num_epochs,
        device=DEVICE,
        save="../../models/multixoz_action_model.pth"
    )
    
else:
    action_model.load_state_dict(torch.load("../../models/multixoz_action_model.pth"))

/tmp/ipykernel_242802/2470509527.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  action_model.load_state_dict(torch.load("../../models/multixoz_action_model.pth"))


RuntimeError: Error(s) in loading state_dict for T5ForConditionalGeneration:
	Missing key(s) in state_dict: "encoder.block.4.layer.0.SelfAttention.q.weight", "encoder.block.4.layer.0.SelfAttention.k.weight", "encoder.block.4.layer.0.SelfAttention.v.weight", "encoder.block.4.layer.0.SelfAttention.o.weight", "encoder.block.4.layer.0.layer_norm.weight", "encoder.block.4.layer.1.DenseReluDense.wi.weight", "encoder.block.4.layer.1.DenseReluDense.wo.weight", "encoder.block.4.layer.1.layer_norm.weight", "encoder.block.5.layer.0.SelfAttention.q.weight", "encoder.block.5.layer.0.SelfAttention.k.weight", "encoder.block.5.layer.0.SelfAttention.v.weight", "encoder.block.5.layer.0.SelfAttention.o.weight", "encoder.block.5.layer.0.layer_norm.weight", "encoder.block.5.layer.1.DenseReluDense.wi.weight", "encoder.block.5.layer.1.DenseReluDense.wo.weight", "encoder.block.5.layer.1.layer_norm.weight", "decoder.block.4.layer.0.SelfAttention.q.weight", "decoder.block.4.layer.0.SelfAttention.k.weight", "dec

In [ ]:
generated_action_train = inference_model(
    action_model, 
    tokenizer, 
    train_action_dataset.actions, 
    MAX_LENGTH_ENCODER_ACTION, 
    MAX_LENGTH_DECODER_ACTION, 
    DEVICE,
    batch_size=1024
) 

generated_action_valid = inference_model(
    action_model, 
    tokenizer, 
    valid_action_dataset.actions, 
    MAX_LENGTH_ENCODER_ACTION, 
    MAX_LENGTH_DECODER_ACTION, 
    DEVICE,
    batch_size=1024
) 

In [ ]:
index = 2
inputs = train_action_dataset.inputs[index]

generated_action = inference_model(
    action_model,
    tokenizer,
    inputs,
    MAX_LENGTH_ENCODER_ACTION,
    MAX_LENGTH_DECODER_ACTION,
    DEVICE
)

print("User Inputs:", inputs)
print("Generated Action:", generated_action)
print("True Action:", train_action_dataset.actions[index])

User Inputs: USER: Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel? SYS: Bedouin's phone is 01223367660. As far as hotels go, I recommend the University Arms Hotel in the center of town.
Generated Action: Hotel-Recommend(area=center of town, name=Bedouin's, phone=01223367660) |
True Action: Hotel-Recommend(area=center of town, name=the University Arms Hotel) | Restaurant-Inform(name=Bedouin, phone=01223367660)
